In [5]:
from astropy import units as u
from astropy.coordinates import SkyCoord
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
from scipy.integrate import quad
import multiprocessing as mp
import gc

In [554]:
#Match every HSC to S82
HSC=pd.read_csv('6_24/HSCol.csv')
S82=pd.read_csv('6_24/S82ol.csv')
HSCs=[[] for i in range(360)]
S82s=[[] for i in range(360)]
for i in range(360):
    HSCs[i]=HSC[(HSC.ra>=i)&(HSC.ra<i+1)]
    S82s[i]=S82[(S82.ra>=i)&(S82.ra<i+1)]
def sep_s2m(A,Bs):
    s=SkyCoord(A[0],A[1],unit='deg')
    m=SkyCoord(Bs[:,0],Bs[:,1],unit='deg')
    sep=s.separation(m)
    return sep.arcsec

TF=[[] for i in range(360)]

def calculate(i):
    t=[]
    print(i,'b',flush=True)
    As = HSCs[i].values
    Bs = S82s[i].values
    for j in range(len(HSCs[i])):
        sep = sep_s2m(As[j], Bs)
        m = sep<=1
        t.append([])
        for k in range(len(m)):
            if m[k]:
                t[j].append(k)
    print(i,'e',flush=True)
    return t
def putout(i):
    print(i,'wb',flush=True)
    matchedHSC = pd.DataFrame(columns=HSC.columns)
    matchedS82 = pd.DataFrame(columns=S82.columns)
    unmatchedHSC = pd.DataFrame(columns=HSC.columns)
    matchlist = pd.DataFrame(columns=['HSCindex', 'HSCra','HSCdec','matchednum'])
    for j in range(len(TF[i])):
        n = 0
        hra=HSCs[i].iloc[j].ra
        hdec=HSCs[i].iloc[j].dec
        matchlist = matchlist.append({'HSCindex': int(HSCs[i].iloc[j].name), \
                                      'HSCra': hra, \
                                      'HSCdec' :hdec, \
                                      'HSCgpsf': HSCs[i].iloc[j].g_psfflux_mag, \
                                      'HSCrpsf': HSCs[i].iloc[j].r_psfflux_mag, \
                                      'HSCipsf': HSCs[i].iloc[j].i_psfflux_mag, \
                                      'HSCzpsf': HSCs[i].iloc[j].z_psfflux_mag, \
                                      'HSCgpsferr': HSCs[i].iloc[j].g_psfflux_magsigma, \
                                      'HSCrpsferr': HSCs[i].iloc[j].r_psfflux_magsigma, \
                                      'HSCipsferr': HSCs[i].iloc[j].i_psfflux_magsigma, \
                                      'HSCzpsferr': HSCs[i].iloc[j].z_psfflux_magsigma, \
                                      'HSCg': HSCs[i].iloc[j].g_cmodel_mag, \
                                      'HSCr': HSCs[i].iloc[j].r_cmodel_mag, \
                                      'HSCi': HSCs[i].iloc[j].i_cmodel_mag, \
                                      'HSCz': HSCs[i].iloc[j].z_cmodel_mag, \
                                      'HSCgerr': HSCs[i].iloc[j].g_cmodel_magsigma, \
                                      'HSCrerr': HSCs[i].iloc[j].r_cmodel_magsigma, \
                                      'HSCierr': HSCs[i].iloc[j].i_cmodel_magsigma, \
                                      'HSCzerr': HSCs[i].iloc[j].z_cmodel_magsigma, \}, ignore_index=True)
        for k in range(len(TF[i][j])):
            if TF[i][j]!=[]:
                n = n + 1
                matchedS82 = matchedS82.append(S82s[i].iloc[TF[i][j][k]])
                matchlist.loc[j, 'matchedS82index' + str(n)] = int(S82s[i].iloc[TF[i][j][k]].name)
                sra=S82s[i].iloc[TF[i][j][k]].ra
                sdec=S82s[i].iloc[TF[i][j][k]].dec
                matchlist.loc[j, 'S82ra' + str(n)] = sra
                matchlist.loc[j, 'S82dec' + str(n)] = sdec
                matchlist.loc[j, 'S82sep' + str(n)] = SkyCoord(hra,hdec,unit='deg').separation(SkyCoord(sra,sdec,unit='deg')).arcsec
                matchlist.loc[j, 'S82gpsf' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_g
                matchlist.loc[j, 'S82rpsf' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_r
                matchlist.loc[j, 'S82ipsf' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_i
                matchlist.loc[j, 'S82zpsf' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_z
                matchlist.loc[j, 'S82gpsferr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_g
                matchlist.loc[j, 'S82rpsferr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_r
                matchlist.loc[j, 'S82ipsferr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_i
                matchlist.loc[j, 'S82zpsferr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_z
                matchlist.loc[j, 'S82g' + str(n)] = S82s[i].iloc[TF[i][j][k]].g
                matchlist.loc[j, 'S82r' + str(n)] = S82s[i].iloc[TF[i][j][k]].r
                matchlist.loc[j, 'S82i' + str(n)] = S82s[i].iloc[TF[i][j][k]].i
                matchlist.loc[j, 'S82z' + str(n)] = S82s[i].iloc[TF[i][j][k]].z
                matchlist.loc[j, 'S82gerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].err_g
                matchlist.loc[j, 'S82rerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].err_r
                matchlist.loc[j, 'S82ierr' + str(n)] = S82s[i].iloc[TF[i][j][k]].err_i
                matchlist.loc[j, 'S82zerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].err_z      
        if n != 0:
            matchedHSC = matchedHSC.append(HSCs[i].iloc[j])
        else:
            unmatchedHSC = unmatchedHSC.append(HSCs[i].iloc[j])
        matchlist.loc[j, 'matchednum'] = n
    print(i,'we',flush=True)
    return matchedHSC,matchedS82,unmatchedHSC,matchlist
h=[]
for i in range(360):
    if HSCs[i].empty:
        continue
    else:
        h.append(i)
pool1=mp.Pool(8)
data=pool1.map(calculate,h)
pool1.close()
for i in h:
    TF[i]=data[h.index(i)]
print('calculation finished',flush=True)
pool2=mp.Pool(8)
data=pool2.map(putout,h)
pool2.close()
matchedHSC=[pd.DataFrame(columns=HSC.columns) for i in range(360)]
matchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
unmatchedHSC=[pd.DataFrame(columns=HSC.columns) for i in range(360)]
unmatchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
matchlist=[pd.DataFrame(columns=['HSCindex','matchednum']) for i in range(360)]
for i in h:
    matchedHSC[i]=data[h.index(i)][0]
    matchedS82[i]=data[h.index(i)][1]
    unmatchedHSC[i]=data[h.index(i)][2]
    matchlist[i]=data[h.index(i)][3]
print('output finished',flush=True)
m=pd.concat([matchlist[i] for i in h],sort=False)
mh=pd.concat([matchedHSC[i] for i in h],sort=False)
umh=pd.concat([unmatchedHSC[i] for i in h],sort=False)
ms=pd.concat([matchedS82[i] for i in h],sort=False)
ms=ms.drop_duplicates()
ums=(S82.append(ms)).drop_duplicates(keep=False)
pwd2='6_24/'
mh.to_csv(pwd2+'matchedHSC.csv',index_label='HSCindex')
ms.to_csv(pwd2+'matchedS82.csv',index_label='S82index')
umh.to_csv(pwd2+'unmatchedHSC.csv',index_label='HSCindex')
ums.to_csv(pwd2+'unmatchedS82.csv',index_label='S82index')
m.to_csv(pwd2+'matchlist.csv',index=False)

30 b
2 b
0 b
19 b
28 b
21 b
15 b
17 b
28 e
29 b
15 e
16 b
30 e
31 b
29 e
32 b
31 e
34 b
34 e
35 b
32 e
33 b
35 e
329 b
33 e
331 b
21 e
22 b
19 e
20 b
17 e
18 b
16 e
333 b
329 e
330 b
2 e
3 b
22 e
335 b
0 e
1 b
20 e
337 b
18 e
339 b
3 e
341 b
1 e
343 b
337 e
338 b
341 e
342 b
339 e
340 b
343 e
344 b
331 e
332 b
335 e
336 b
333 e
334 b
330 e
345 b
342 e
347 b
344 e
349 b
340 e
351 b
338 e
353 b
345 e
346 b
351 e
352 b
353 e
354 b
347 e
348 b
349 e
350 b
336 e
355 b
352 e
357 b
354 e
359 b
346 e
332 e
334 e
350 e
355 e
356 b
348 e
357 e
358 b
359 e
356 e
358 e
calculation finished
0 wb
2 wb
17 wb
19 wb
21 wb
15 wb
28 wb
30 wb
28 we
29 wb
15 we
16 wb
30 we
31 wb
29 we
32 wb
31 we
34 wb
21 we
22 wb
32 we
33 wb
34 we
35 wb
19 we
20 wb
33 we
329 wb
16 we
331 wb
17 we
18 wb
35 we
333 wb
22 we
335 wb
2 we
3 wb
0 we
1 wb
329 we
330 wb
20 we
337 wb
18 we
339 wb
3 we
341 wb
1 we
343 wb
335 we
336 wb
341 we
342 wb
339 we
340 wb
337 we
338 wb
331 we
332 wb
333 we
334 wb
330 we
345 wb
343 we
344 wb
3

In [551]:
#match every HSC galaxy to unmatched S82
HSC=pd.read_csv('6_23/glol.csv')
S82=pd.read_csv('6_23/unmatchedS82.csv')
HSCs=[[] for i in range(360)]
S82s=[[] for i in range(360)]
for i in range(360):
    HSCs[i]=HSC[(HSC.ra>=i)&(HSC.ra<i+1)]
    S82s[i]=S82[(S82.ra>=i)&(S82.ra<i+1)]
def sep_s2m(A,Bs):
    s=SkyCoord(A[0],A[1],unit='deg')
    m=SkyCoord(Bs[:,1],Bs[:,2],unit='deg')
    sep=s.separation(m)
    return sep.arcsec
TF=[[] for i in range(360)]

def calculate(i):
    t=[]
    print(i,'b',flush=True)
    As = HSCs[i].values
    Bs = S82s[i].values
    for j in range(len(HSCs[i])):
        sep = sep_s2m(As[j], Bs)
        m = sep<=1
        t.append([])
        for k in range(len(m)):
            if m[k]:
                t[j].append(k)
    print(i,'e',flush=True)
    return t
def putout(i):
    print(i,'wb',flush=True)
    matchedHSC = pd.DataFrame(columns=HSC.columns)
    matchedS82 = pd.DataFrame(columns=S82.columns)
    unmatchedHSC = pd.DataFrame(columns=HSC.columns)
    matchlist = pd.DataFrame(columns=['HSCindex', 'HSCra','HSCdec','matchednum'])
    for j in range(len(TF[i])):
        n = 0
        hra=HSCs[i].iloc[j].ra
        hdec=HSCs[i].iloc[j].dec
        matchlist = matchlist.append({'HSCindex': int(HSCs[i].iloc[j].name), 'HSCra': hra,'HSCdec' :hdec, 'HSCgmag': HSCs[i].iloc[j].g_psfflux_mag,'HSCrmag': HSCs[i].iloc[j].r_psfflux_mag,'HSCimag': HSCs[i].iloc[j].i_psfflux_mag,'HSCzmag': HSCs[i].iloc[j].z_psfflux_mag, 'HSCgmagerr': HSCs[i].iloc[j].g_psfflux_magsigma, 'HSCrmagerr': HSCs[i].iloc[j].r_psfflux_magsigma,'HSCimagerr':HSCs[i].iloc[j].i_psfflux_magsigma,'HSCzmagerr':HSCs[i].iloc[j].z_psfflux_magsigma}, ignore_index=True)
        for k in range(len(TF[i][j])):
            if TF[i][j]!=[]:
                n = n + 1
                matchedS82 = matchedS82.append(S82s[i].iloc[TF[i][j][k]])
                matchlist.loc[j, 'matchedS82index' + str(n)] = int(S82s[i].iloc[TF[i][j][k]].S82index)
                sra=S82s[i].iloc[TF[i][j][k]].ra
                sdec=S82s[i].iloc[TF[i][j][k]].dec
                matchlist.loc[j, 'S82ra' + str(n)] = sra
                matchlist.loc[j, 'S82dec' + str(n)] = sdec
                matchlist.loc[j, 'S82sep' + str(n)] = SkyCoord(hra,hdec,unit='deg').separation(SkyCoord(sra,sdec,unit='deg')).arcsec
                matchlist.loc[j, 'S82gmag' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_g
                matchlist.loc[j, 'S82rmag' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_r
                matchlist.loc[j, 'S82imag' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_i
                matchlist.loc[j, 'S82zmag' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_z
                matchlist.loc[j, 'S82gmagerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_g
                matchlist.loc[j, 'S82rmagerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_r
                matchlist.loc[j, 'S82imagerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_i
                matchlist.loc[j, 'S82zmagerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_z    
        if n != 0:
            matchedHSC = matchedHSC.append(HSCs[i].iloc[j])
        else:
            unmatchedHSC = unmatchedHSC.append(HSCs[i].iloc[j])
        matchlist.loc[j, 'matchednum'] = n
    print(i,'we',flush=True)
    return matchedHSC,matchedS82,unmatchedHSC,matchlist
h=[]
for i in range(360):
    if HSCs[i].empty:
        continue
    else:
        h.append(i)
pool1=mp.Pool(8)
data=pool1.map(calculate,h)
pool1.close()
for i in h:
    TF[i]=data[h.index(i)]
print('calculation finished',flush=True)
pool2=mp.Pool(8)
data=pool2.map(putout,h)
pool2.close()
matchedHSC=[pd.DataFrame(columns=HSC.columns) for i in range(360)]
matchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
unmatchedHSC=[pd.DataFrame(columns=HSC.columns) for i in range(360)]
unmatchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
matchlist=[pd.DataFrame(columns=['HSCindex','matchednum']) for i in range(360)]
for i in h:
    matchedHSC[i]=data[h.index(i)][0]
    matchedS82[i]=data[h.index(i)][1]
    unmatchedHSC[i]=data[h.index(i)][2]
    matchlist[i]=data[h.index(i)][3]
print('output finished',flush=True)
m=pd.concat([matchlist[i] for i in h],sort=False)
mh=pd.concat([matchedHSC[i] for i in h],sort=False)
umh=pd.concat([unmatchedHSC[i] for i in h],sort=False)
ms=pd.concat([matchedS82[i] for i in h],sort=False)
ms=ms.drop_duplicates()
ums=(S82.append(ms)).drop_duplicates(keep=False)
pwd2='6_17/'
mh.to_csv(pwd2+'glmatchedHSC.csv',index_label='HSCindex')
ms.to_csv(pwd2+'glmatchedS82.csv',index_label='S82index')
umh.to_csv(pwd2+'glunmatchedHSC.csv',index_label='HSCindex')
ums.to_csv(pwd2+'glunmatchedS82.csv',index=False)
m.to_csv(pwd2+'glmatchlist.csv',index=False)

0 b
15 b
2 b
17 b
19 b
21 b
28 b
30 b
28 e
29 b
30 e
31 b
15 e
16 b
29 e
32 b
31 e
34 b
34 e
35 b
32 e
33 b
35 e
329 b
33 e
331 b
2 e
3 b
0 e
1 b
21 e
22 b
329 e
330 b
19 e
20 b
17 e
18 b
16 e
333 b
3 e
335 b
22 e
337 b
331 e
332 b
1 e
339 b
20 e
341 b
330 e
343 b
18 e
345 b
337 e
338 b
333 e
334 b
343 e
344 b
332 e
347 b
335 e
336 b
339 e
340 b
341 e
342 b
334 e
349 b
344 e
351 b
345 e
346 b
338 e
353 b
342 e
355 b
347 e
348 b
336 e
357 b
340 e
359 b
351 e
352 b
349 e
350 b
348 e
353 e
354 b
346 e
357 e
358 b
359 e
355 e
356 b
352 e
350 e
354 e
358 e
356 e
calculation finished
0 wb
19 wb
21 wb
28 wb
30 wb
2 wb
17 wb
15 wb
28 we
29 wb
15 we
16 wb
30 we
31 wb
29 we
32 wb
21 we
22 wb
31 we
34 wb
34 we
35 wb
32 we
33 wb
35 we
329 wb
19 we
20 wb
33 we
331 wb
329 we
330 wb
22 we
333 wb
16 we
335 wb
2 we
3 wb
17 we
18 wb
0 we
1 wb
20 we
337 wb
331 we
332 wb
330 we
339 wb
3 we
341 wb
333 we
334 wb
18 we
343 wb
337 we
338 wb
335 we
336 wb
339 we
340 wb
1 we
345 wb
332 we
347 wb
341 we
342 wb
3

In [4]:
#match every unmatched S82 to HSC galaxy
HSC=pd.read_csv('6_21/glol.csv')
S82=pd.read_csv('6_21/unmatchedS82.csv')
HSCs=[[] for i in range(360)]
S82s=[[] for i in range(360)]
for i in range(360):
    HSCs[i]=HSC[(HSC.ra>=i)&(HSC.ra<i+1)]
    S82s[i]=S82[(S82.ra>=i)&(S82.ra<i+1)]

def sep_s2m(A,Bs):
    s=SkyCoord(A[1],A[2],unit='deg')
    m=SkyCoord(Bs[:,0],Bs[:,1],unit='deg')
    sep=s.separation(m)
    return sep.arcsec


def calculate(i):
    t=[]
    print(i,'b',flush=True)
    As = S82s[i].values
    Bs = HSCs[i].values
    for j in range(len(S82s[i])):
        sep = sep_s2m(As[j], Bs)
        m = sep<=1
        t.append([])
        for k in range(len(m)):
            if m[k]:
                t[j].append(k)
    print(i,'e',flush=True)
    return t

def putout(i):
    print(i,'wb',flush=True)
    matchedHSC = pd.DataFrame(columns=HSC.columns)
    matchedS82 = pd.DataFrame(columns=S82.columns)
    unmatchedS82 = pd.DataFrame(columns=S82.columns)
    matchlist = pd.DataFrame(columns=['S82index', 'S82ra','S82dec','matchednum'])
    for j in range(len(TF[i])):
        n = 0
        hra=S82s[i].iloc[j].ra
        hdec=S82s[i].iloc[j].dec
        matchlist = matchlist.append({'S82index': int(S82s[i].iloc[j].S82index), \
                                      'S82ra': hra, \
                                      'S82dec' :hdec, \
                                      'S82gpsf': S82s[i].iloc[j].psfMag_g, \
                                      'S82rpsf': S82s[i].iloc[j].psfMag_r, \
                                      'S82ipsf': S82s[i].iloc[j].psfMag_i, \
                                      'S82zpsf': S82s[i].iloc[j].psfMag_z, \
                                      'S82gpsferr': S82s[i].iloc[j].psfmagerr_g, \
                                      'S82rpsferr': S82s[i].iloc[j].psfmagerr_r, \
                                      'S82ipsferr': S82s[i].iloc[j].psfmagerr_i, \
                                      'S82zpsferr': S82s[i].iloc[j].psfmagerr_z, \
                                      'S82g': S82s[i].iloc[j].g, \
                                      'S82r': S82s[i].iloc[j].r, \
                                      'S82i': S82s[i].iloc[j].i, \
                                      'S82z': S82s[i].iloc[j].z, \
                                      'S82gerr': S82s[i].iloc[j].err_g, \
                                      'S82rerr': S82s[i].iloc[j].err_r, \
                                      'S82ierr': S82s[i].iloc[j].err_i, \
                                      'S82zerr': S82s[i].iloc[j].err_z, \
                                     }, ignore_index=True)
        if TF[i][j]!=[]:
            for k in range(len(TF[i][j])):
                n = n + 1
                matchedHSC = matchedHSC.append(HSCs[i].iloc[TF[i][j][k]])
                matchlist.loc[j, 'matchedHSCindex' + str(n)] = int(HSCs[i].iloc[TF[i][j][k]].name)
                sra=HSCs[i].iloc[TF[i][j][k]].ra
                sdec=HSCs[i].iloc[TF[i][j][k]].dec
                matchlist.loc[j, 'HSCra' + str(n)] = sra
                matchlist.loc[j, 'HSCdec' + str(n)] = sdec
                matchlist.loc[j, 'HSCsep' + str(n)] = SkyCoord(hra,hdec,unit='deg').separation(SkyCoord(sra,sdec,unit='deg')).arcsec
                matchlist.loc[j, 'HSCgpsf' + str(n)] = HSCs[i].iloc[TF[i][j][k]].g_psfflux_mag
                matchlist.loc[j, 'HSCrpsf' + str(n)] = HSCs[i].iloc[TF[i][j][k]].r_psfflux_mag
                matchlist.loc[j, 'HSCipsf' + str(n)] = HSCs[i].iloc[TF[i][j][k]].i_psfflux_mag
                matchlist.loc[j, 'HSCzpsf' + str(n)] = HSCs[i].iloc[TF[i][j][k]].z_psfflux_mag
                matchlist.loc[j, 'HSCgpsferr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].g_psfflux_magsigma
                matchlist.loc[j, 'HSCrpsferr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].r_psfflux_magsigma
                matchlist.loc[j, 'HSCipsferr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].i_psfflux_magsigma
                matchlist.loc[j, 'HSCzpsferr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].z_psfflux_magsigma
                matchlist.loc[j, 'HSCg' + str(n)] = HSCs[i].iloc[TF[i][j][k]].g_cmodel_mag
                matchlist.loc[j, 'HSCr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].r_cmodel_mag
                matchlist.loc[j, 'HSCi' + str(n)] = HSCs[i].iloc[TF[i][j][k]].i_cmodel_mag
                matchlist.loc[j, 'HSCz' + str(n)] = HSCs[i].iloc[TF[i][j][k]].z_cmodel_mag
                matchlist.loc[j, 'HSCgerr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].g_cmodel_magsigma
                matchlist.loc[j, 'HSCrerr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].r_cmodel_magsigma
                matchlist.loc[j, 'HSCierr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].i_cmodel_magsigma
                matchlist.loc[j, 'HSCzerr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].z_cmodel_magsigma   
        if n != 0:      
            matchedS82 = matchedS82.append(S82s[i].iloc[j])
        else:
            unmatchedS82 = unmatchedS82.append(S82s[i].iloc[j])
        matchlist.loc[j, 'matchednum'] = n
    print(i,'we',flush=True)
    return matchedHSC,matchedS82,unmatchedS82,matchlist

TF=[[] for i in range(360)]

h=[]
for i in range(360):
    if S82s[i].empty:
        continue
    else:
        h.append(i)
pool1=mp.Pool(8)
data=pool1.map(calculate,h)
pool1.close()
for i in h:
    TF[i]=data[h.index(i)]
print('calculation finished',flush=True)

pool2=mp.Pool(8)
data2=pool2.map(putout,h)
pool2.close()
matchedHSC=[pd.DataFrame(columns=HSC.columns) for i in range(360)]
matchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
unmatchedHSC=[pd.DataFrame(columns=HSC.columns) for i in range(360)]
unmatchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
matchlist=[pd.DataFrame(columns=['HSCindex','matchednum']) for i in range(360)]
for i in h:
    matchedHSC[i]=data2[h.index(i)][0]
    matchedS82[i]=data2[h.index(i)][1]
    unmatchedS82[i]=data2[h.index(i)][2]
    matchlist[i]=data2[h.index(i)][3]
print('output finished',flush=True)
m=pd.concat([matchlist[i] for i in h],sort=False)
mh=pd.concat([matchedHSC[i] for i in h],sort=False)
ums=pd.concat([unmatchedS82[i] for i in h],sort=False)
ms=pd.concat([matchedS82[i] for i in h],sort=False)
mh=mh.drop_duplicates()
umh=(HSC.append(mh)).drop_duplicates(keep=False)
pwd2='6_21/'
mh.to_csv(pwd2+'sglmatchedHSC.csv',index_label='HSCindex')
ms.to_csv(pwd2+'sglmatchedS82.csv',index_label='S82index')
umh.to_csv(pwd2+'sglunmatchedHSC.csv',index_label='HSCindex')
ums.to_csv(pwd2+'sglunmatchedS82.csv',index=False)
m.to_csv(pwd2+'sglmatchlist.csv',index=False)


17 wb
15 wb
0 wb
2 wb
19 wb
21 wb
30 wb
28 wb
28 we
29 wb
30 we
31 wb
29 we
32 wb
31 we
34 wb
21 we
22 wb
34 we
35 wb
19 we
20 wb
15 we
16 wb
2 we
3 wb
32 we
33 wb
17 we
18 wb
0 we
1 wb
35 we
329 wb
33 we
331 wb
22 we
333 wb
20 we
335 wb
18 we
337 wb
16 we
339 wb
1 we
341 wb
3 we
343 wb
331 we
332 wb
343 we
344 wb
333 we
334 wb
337 we
338 wb
339 we
340 wb
335 we
336 wb
341 we
342 wb
332 we
345 wb
344 we
347 wb
334 we
349 wb
338 we
351 wb
342 we
353 wb
340 we
355 wb
329 we
330 wb
336 we
357 wb
345 we
346 wb
347 we
348 wb
349 we
350 wb
351 we
352 wb
353 we
354 wb
355 we
356 wb
357 we
358 wb
330 we
359 wb
346 we
348 we
350 we
352 we
354 we
356 we
358 we
359 we
output finished


In [111]:
#matched gaia to S82
g=pd.read_csv('gaia/gaiaolS82.csv')
S82=pd.read_csv('gaia/S82ol2.csv')
gs=[[] for i in range(360)]
S82s=[[] for i in range(360)]
for i in range(360):
    gs[i]=g[(g.ra>=i)&(g.ra<i+1)]
    S82s[i]=S82[(S82.ra>=i)&(S82.ra<i+1)]
def sep_s2m(A,Bs):
    s=SkyCoord(A[1],A[2],unit='deg')
    m=SkyCoord(Bs[:,0],Bs[:,1],unit='deg')
    sep=s.separation(m)
    return sep.arcsec

TF=[[] for i in range(360)]

def calculate(i):
    t=[]
    print(i,'b',flush=True)
    As = gs[i].values
    Bs = S82s[i].values
    for j in range(len(gs[i])):
        sep = sep_s2m(As[j], Bs)
        m = sep<=1
        t.append([])
        for k in range(len(m)):
            if m[k]:
                t[j].append(k)
    print(i,'e',flush=True)
    return t
def putout(i):
    print(i,'wb',flush=True)
    matchedg = pd.DataFrame(columns=g.columns)
    matchedS82 = pd.DataFrame(columns=S82.columns)
    unmatchedg = pd.DataFrame(columns=g.columns)
    matchlist = pd.DataFrame(columns=['gindex', 'gra','gdec','matchednum'])
    for j in range(len(TF[i])):
        n = 0
        hra=gs[i].iloc[j].ra
        hdec=gs[i].iloc[j].dec
        matchlist = matchlist.append({'gindex': int(gs[i].iloc[j].name), \
                                      'gra': hra, \
                                      'gdec' :hdec, \
                                      gs[i].columns[3] : gs[i][gs[i].columns[3]].iloc[j], \
                                      gs[i].columns[4] : gs[i][gs[i].columns[4]].iloc[j], \
                                      gs[i].columns[5] : gs[i][gs[i].columns[5]].iloc[j] }, ignore_index=True)
        for k in range(len(TF[i][j])):
            if TF[i][j]!=[]:
                n = n + 1
                matchedS82 = matchedS82.append(S82s[i].iloc[TF[i][j][k]])
                matchlist.loc[j, 'matchedS82index' + str(n)] = int(S82s[i].iloc[TF[i][j][k]].name)
                sra=S82s[i].iloc[TF[i][j][k]].ra
                sdec=S82s[i].iloc[TF[i][j][k]].dec
                matchlist.loc[j, 'S82ra' + str(n)] = sra
                matchlist.loc[j, 'S82dec' + str(n)] = sdec
                matchlist.loc[j, 'S82sep' + str(n)] = SkyCoord(hra,hdec,unit='deg').separation(SkyCoord(sra,sdec,unit='deg')).arcsec
                matchlist.loc[j, 'S82gpsf' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_g
                matchlist.loc[j, 'S82rpsf' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_r
                matchlist.loc[j, 'S82ipsf' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_i
                matchlist.loc[j, 'S82zpsf' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfMag_z
                matchlist.loc[j, 'S82gpsferr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_g
                matchlist.loc[j, 'S82rpsferr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_r
                matchlist.loc[j, 'S82ipsferr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_i
                matchlist.loc[j, 'S82zpsferr' + str(n)] = S82s[i].iloc[TF[i][j][k]].psfmagerr_z
                matchlist.loc[j, 'S82g' + str(n)] = S82s[i].iloc[TF[i][j][k]].g
                matchlist.loc[j, 'S82r' + str(n)] = S82s[i].iloc[TF[i][j][k]].r
                matchlist.loc[j, 'S82i' + str(n)] = S82s[i].iloc[TF[i][j][k]].i
                matchlist.loc[j, 'S82z' + str(n)] = S82s[i].iloc[TF[i][j][k]].z
                matchlist.loc[j, 'S82gerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].err_g
                matchlist.loc[j, 'S82rerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].err_r
                matchlist.loc[j, 'S82ierr' + str(n)] = S82s[i].iloc[TF[i][j][k]].err_i
                matchlist.loc[j, 'S82zerr' + str(n)] = S82s[i].iloc[TF[i][j][k]].err_z      
        if n != 0:
            matchedg = matchedg.append(gs[i].iloc[j])
        else:
            unmatchedg = unmatchedg.append(gs[i].iloc[j])
        matchlist.loc[j, 'matchednum'] = n
    print(i,'we',flush=True)
    return matchedg,matchedS82,unmatchedg,matchlist
h=[]
for i in range(360):
    if gs[i].empty:
        continue
    else:
        h.append(i)
pool1=mp.Pool(8)
data=pool1.map(calculate,h)
pool1.close()
for i in h:
    TF[i]=data[h.index(i)]
print('calculation finished',flush=True)
pool2=mp.Pool(8)
data=pool2.map(putout,h)
pool2.close()
matchedg=[pd.DataFrame(columns=g.columns) for i in range(360)]
matchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
unmatchedg=[pd.DataFrame(columns=g.columns) for i in range(360)]
unmatchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
matchlist=[pd.DataFrame(columns=['gindex','matchednum']) for i in range(360)]
for i in h:
    matchedg[i]=data[h.index(i)][0]
    matchedS82[i]=data[h.index(i)][1]
    unmatchedg[i]=data[h.index(i)][2]
    matchlist[i]=data[h.index(i)][3]
print('output finished',flush=True)
m=pd.concat([matchlist[i] for i in h],sort=False)
mg=pd.concat([matchedg[i] for i in h],sort=False)
umg=pd.concat([unmatchedg[i] for i in h],sort=False)
ms=pd.concat([matchedS82[i] for i in h],sort=False)
ms=ms.drop_duplicates()
ums=(S82.append(ms)).drop_duplicates(keep=False)
pwd2='gaia/'
mg.to_csv(pwd2+'matchedg.csv',index_label='gindex')
ms.to_csv(pwd2+'matchedS82.csv',index_label='S82index')
umg.to_csv(pwd2+'unmatchedg.csv',index_label='gindex')
ums.to_csv(pwd2+'unmatchedS82.csv',index_label='S82index')
m.to_csv(pwd2+'matchlist.csv',index=False)

2 b
0 b
17 b
15 b
19 b
21 b
28 b
30 b
28 e
29 b
30 e
31 b
29 e
32 b
31 e
34 b
34 e
35 b
32 e
33 b
35 e
329 b
33 e
331 b
15 e
16 b
21 e
22 b
19 e
20 b
17 e
18 b
2 e
3 b
0 e
1 b
22 e
333 b
20 e
335 b
16 e
337 b
18 e
339 b
3 e
341 b
1 e
343 b
329 e
330 b
339 e
340 b
343 e
344 b
341 e
342 b
337 e
338 b
331 e
332 b
335 e
336 b
333 e
334 b
344 e
345 b
342 e
347 b
340 e
349 b
338 e
351 b
330 e
353 b
345 e
346 b
349 e
350 b
347 e
348 b
332 e
355 b
351 e
352 b
336 e
357 b
353 e
354 b
334 e
359 b
350 e
348 e
355 e
356 b
346 e
357 e
358 b
352 e
354 e
359 e
356 e
358 e
calculation finished
0 wb
2 wb
15 wb
19 wb
17 wb
28 wb
21 wb
30 wb
28 we
29 wb
30 we
31 wb
29 we
32 wb
31 we
34 wb
34 we
35 wb
32 we
33 wb
15 we
16 wb
21 we
22 wb
19 we
20 wb
17 we
18 wb
2 we
3 wb
35 we
329 wb
33 we
331 wb
0 we
1 wb
22 we
333 wb
20 we
335 wb
16 we
337 wb
18 we
339 wb
3 we
341 wb
1 we
343 wb
329 we
330 wb
331 we
332 wb
339 we
340 wb
335 we
336 wb
337 we
338 wb
343 we
344 wb
341 we
342 wb
333 we
334 wb
330 we
345 wb
3

In [ ]:
#match every quasar to HSC all
q=pd.read_csv('quasar/quasar.csv')
hsc=pd.read_csv('quasar/248491.csv')
qs=[[] for i in range(360)]
hscs=[[] for i in range(360)]
for i in range(360):
    HSCs[i]=HSC[(HSC.ra>=i)&(HSC.ra<i+1)]
    S82s[i]=S82[(S82.ra>=i)&(S82.ra<i+1)]

def sep_s2m(A,Bs):
    s=SkyCoord(A[1],A[2],unit='deg')
    m=SkyCoord(Bs[:,0],Bs[:,1],unit='deg')
    sep=s.separation(m)
    return sep.arcsec


def calculate(i):
    t=[]
    print(i,'b',flush=True)
    As = S82s[i].values
    Bs = HSCs[i].values
    for j in range(len(S82s[i])):
        sep = sep_s2m(As[j], Bs)
        m = sep<=1
        t.append([])
        for k in range(len(m)):
            if m[k]:
                t[j].append(k)
    print(i,'e',flush=True)
    return t

def putout(i):
    print(i,'wb',flush=True)
    matchedHSC = pd.DataFrame(columns=HSC.columns)
    matchedS82 = pd.DataFrame(columns=S82.columns)
    unmatchedS82 = pd.DataFrame(columns=S82.columns)
    matchlist = pd.DataFrame(columns=['S82index', 'S82ra','S82dec','matchednum'])
    for j in range(len(TF[i])):
        n = 0
        hra=S82s[i].iloc[j].ra
        hdec=S82s[i].iloc[j].dec
        matchlist = matchlist.append({'S82index': int(S82s[i].iloc[j].S82index), \
                                      'S82ra': hra, \
                                      'S82dec' :hdec, \
                                      'S82gpsf': S82s[i].iloc[j].psfMag_g, \
                                      'S82rpsf': S82s[i].iloc[j].psfMag_r, \
                                      'S82ipsf': S82s[i].iloc[j].psfMag_i, \
                                      'S82zpsf': S82s[i].iloc[j].psfMag_z, \
                                      'S82gpsferr': S82s[i].iloc[j].psfmagerr_g, \
                                      'S82rpsferr': S82s[i].iloc[j].psfmagerr_r, \
                                      'S82ipsferr': S82s[i].iloc[j].psfmagerr_i, \
                                      'S82zpsferr': S82s[i].iloc[j].psfmagerr_z, \
                                      'S82g': S82s[i].iloc[j].g, \
                                      'S82r': S82s[i].iloc[j].r, \
                                      'S82i': S82s[i].iloc[j].i, \
                                      'S82z': S82s[i].iloc[j].z, \
                                      'S82gerr': S82s[i].iloc[j].err_g, \
                                      'S82rerr': S82s[i].iloc[j].err_r, \
                                      'S82ierr': S82s[i].iloc[j].err_i, \
                                      'S82zerr': S82s[i].iloc[j].err_z, \
                                     }, ignore_index=True)
        if TF[i][j]!=[]:
            for k in range(len(TF[i][j])):
                n = n + 1
                matchedHSC = matchedHSC.append(HSCs[i].iloc[TF[i][j][k]])
                matchlist.loc[j, 'matchedHSCindex' + str(n)] = int(HSCs[i].iloc[TF[i][j][k]].name)
                sra=HSCs[i].iloc[TF[i][j][k]].ra
                sdec=HSCs[i].iloc[TF[i][j][k]].dec
                matchlist.loc[j, 'HSCra' + str(n)] = sra
                matchlist.loc[j, 'HSCdec' + str(n)] = sdec
                matchlist.loc[j, 'HSCsep' + str(n)] = SkyCoord(hra,hdec,unit='deg').separation(SkyCoord(sra,sdec,unit='deg')).arcsec
                matchlist.loc[j, 'HSCgpsf' + str(n)] = HSCs[i].iloc[TF[i][j][k]].g_psfflux_mag
                matchlist.loc[j, 'HSCrpsf' + str(n)] = HSCs[i].iloc[TF[i][j][k]].r_psfflux_mag
                matchlist.loc[j, 'HSCipsf' + str(n)] = HSCs[i].iloc[TF[i][j][k]].i_psfflux_mag
                matchlist.loc[j, 'HSCzpsf' + str(n)] = HSCs[i].iloc[TF[i][j][k]].z_psfflux_mag
                matchlist.loc[j, 'HSCgpsferr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].g_psfflux_magsigma
                matchlist.loc[j, 'HSCrpsferr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].r_psfflux_magsigma
                matchlist.loc[j, 'HSCipsferr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].i_psfflux_magsigma
                matchlist.loc[j, 'HSCzpsferr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].z_psfflux_magsigma
                matchlist.loc[j, 'HSCg' + str(n)] = HSCs[i].iloc[TF[i][j][k]].g_cmodel_mag
                matchlist.loc[j, 'HSCr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].r_cmodel_mag
                matchlist.loc[j, 'HSCi' + str(n)] = HSCs[i].iloc[TF[i][j][k]].i_cmodel_mag
                matchlist.loc[j, 'HSCz' + str(n)] = HSCs[i].iloc[TF[i][j][k]].z_cmodel_mag
                matchlist.loc[j, 'HSCgerr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].g_cmodel_magsigma
                matchlist.loc[j, 'HSCrerr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].r_cmodel_magsigma
                matchlist.loc[j, 'HSCierr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].i_cmodel_magsigma
                matchlist.loc[j, 'HSCzerr' + str(n)] = HSCs[i].iloc[TF[i][j][k]].z_cmodel_magsigma   
        if n != 0:      
            matchedS82 = matchedS82.append(S82s[i].iloc[j])
        else:
            unmatchedS82 = unmatchedS82.append(S82s[i].iloc[j])
        matchlist.loc[j, 'matchednum'] = n
    print(i,'we',flush=True)
    return matchedHSC,matchedS82,unmatchedS82,matchlist

TF=[[] for i in range(360)]

h=[]
for i in range(360):
    if S82s[i].empty:
        continue
    else:
        h.append(i)
pool1=mp.Pool(8)
data=pool1.map(calculate,h)
pool1.close()
for i in h:
    TF[i]=data[h.index(i)]
print('calculation finished',flush=True)

pool2=mp.Pool(8)
data2=pool2.map(putout,h)
pool2.close()
matchedHSC=[pd.DataFrame(columns=HSC.columns) for i in range(360)]
matchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
unmatchedHSC=[pd.DataFrame(columns=HSC.columns) for i in range(360)]
unmatchedS82=[pd.DataFrame(columns=S82.columns) for i in range(360)]
matchlist=[pd.DataFrame(columns=['HSCindex','matchednum']) for i in range(360)]
for i in h:
    matchedHSC[i]=data2[h.index(i)][0]
    matchedS82[i]=data2[h.index(i)][1]
    unmatchedS82[i]=data2[h.index(i)][2]
    matchlist[i]=data2[h.index(i)][3]
print('output finished',flush=True)
m=pd.concat([matchlist[i] for i in h],sort=False)
mh=pd.concat([matchedHSC[i] for i in h],sort=False)
ums=pd.concat([unmatchedS82[i] for i in h],sort=False)
ms=pd.concat([matchedS82[i] for i in h],sort=False)
mh=mh.drop_duplicates()
umh=(HSC.append(mh)).drop_duplicates(keep=False)
pwd2='6_21/'
mh.to_csv(pwd2+'sglmatchedHSC.csv',index_label='HSCindex')
ms.to_csv(pwd2+'sglmatchedS82.csv',index_label='S82index')
umh.to_csv(pwd2+'sglunmatchedHSC.csv',index_label='HSCindex')
ums.to_csv(pwd2+'sglunmatchedS82.csv',index=False)
m.to_csv(pwd2+'sglmatchlist.csv',index=False)

